# Import Libraries

In [ ]:
pip install -U layer

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error as mae
import layer
from layer.decorators import dataset,model, pip_requirements

## Log in to your Layer account

In [3]:
layer.login()

Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://app.layer.ai/oauth/authorize?response_type=code&code_challenge=ndD_xwqkZU3c3Nj3KsvknGN5kJRA58m_sFCKX7BdCKw&code_challenge_method=S256&client_id=0STDdcnpK48P8A429EAAn93WNuLmViLR&redirect_uri=https://app.layer.ai/oauth/code&scope=offline_access&audience=https://app.layer.ai
Code: _s9Bk1AWB9OSKSQTaL6dLLxNwe5FjWf_JvtYlEDuNB9yG
Successfully logged into https://app.layer.ai


## Create a Layer project

In [13]:
layer.init("Darius_airqo")

Your Layer project is here: https://app.layer.ai/darius/Darius_airqo

## Fetch the datasets as Pandas DataFrames

In [14]:
ss = layer.get_dataset("zindi/air-quality/datasets/sample_submission").to_pandas()
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()

Output()

Output()

Output()

In [15]:
target="pm2_5"

In [16]:
# Drop Correlared Cols
cc=['site_longitude','SulphurDioxide_SO2_column_number_density','Cloud_cloud_base_height','NitrogenDioxide_sensor_altitude','SulphurDioxide_sensor_azimuth_angle','SulphurDioxide_sensor_zenith_angle',
    'NitrogenDioxide_sensor_azimuth_angle','NitrogenDioxide_sensor_zenith_angle','Formaldehyde_sensor_azimuth_angle','Formaldehyde_sensor_zenith_angle','Ozone_sensor_azimuth_angle','Ozone_sensor_zenith_angle',
    'Cloud_sensor_azimuth_angle','Cloud_sensor_zenith_angle','CarbonMonoxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle']
train_df.drop(cc,axis=1,inplace=True)
#Preprocess date
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
# density features
density_amf = list(train_df.filter(regex='density_amf').columns)
density = ['SulphurDioxide_SO2_slant_column_number_density','SulphurDioxide_SO2_column_number_density_15km','CarbonMonoxide_CO_column_number_density','CarbonMonoxide_H2O_column_number_density',
 'NitrogenDioxide_NO2_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'NitrogenDioxide_NO2_slant_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density',
 'Formaldehyde_HCHO_slant_column_number_density','Ozone_O3_column_number_density','Ozone_O3_slant_column_number_density']
# Feature Clusters
Cloud = list(train_df.filter(regex='Cloud').columns)
Ozone = list(train_df.filter(regex='Ozone').columns)
Carbon = list(train_df.filter(regex='Carbon').columns)
Aerosol = list(train_df.filter(regex='Aerosol').columns)
Sulphur = list(train_df.filter(regex='Sulphur').columns)
Nitrogen = list(train_df.filter(regex='Nitrogen').columns)
Formaldehyde = list(train_df.filter(regex='Formaldehyde').columns)
Weather = ['temp_mean','humidity']
test_df[target]=np.nan
test_df=test_df[train_df.columns]
train_df=train_df.append(test_df)
train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# Feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    for i in range(1,26,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]
# new devices
missing=train_df[train_df.device.isin(['aq_91', 'aq_98'])]
le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

length of features =  1536


## Model training with Layer

In [18]:
!pip install catboost -q
from catboost import CatBoostRegressor 

**First Model**

In [23]:
@model("model1")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
# model1 = layer.get_model("tomy4reel/aqssh-645h-wver/models/model1").get_train()
# submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
# submission["pm2_5"] = model1.predict(test_df[cols])    

Output()

08:50:28 model1: Successfully logged into https://app.layer.ai

08:50:30 model1: Defaulting to user installation because normal site-packages is not writeable

08:50:30 model1: Collecting catboost

08:50:31 model1:   Downloading catboost-1.0.6-cp37-none-manylinux1_x86_64.whl (76.6 MB)

...

❌ Project execution error.

🔎 Train 831cd731-10b6-4331-b476-4574e660c5b2 failed with "cxxrt::bad_alloc"

💡 Add `debug=True` parameter to your `layer.run()` to see server logs

08:50:52 model1: Traceback (most recent call last):
08:50:52 model1:   File "/home/nonroot/.local/lib/python3.7/site-packages/layer/exceptions/exception_handler.py", line 31, in wrapper
08:50:52 model1:     return wrapped(*args, **kwargs)
08:50:52 model1:   File "/home/nonroot/.local/lib/python3.7/site-packages/layer/training/runtime/model_trainer.py", line 184, in _train
08:50:52 model1:     model = train_model_func()
08:50:52 model1:   File "/usr/local/lib/python3.7/dist-packages/layer/decorators/pip_requirements_decorator.py", line 44, in wrapper
08:50:52 model1:   File "<ipython-input-23-8c10c369afa1>", line 13, in train
08:50:52 model1:   File "/home/nonroot/.local/lib/python3.7/site-packages/catboost/core.py", line 5593, in fit
08:50:52 model1:     save_snapshot, snapshot_file, snapshot_interval, init_model, callbacks, log_cout, log_cerr)
08:50:52 model1:   File "/home/nonroot/.local/lib/python3.7/site-packages/catboost/core.py", line 2283, in _fit
08:50:52 model1:     train_para

ProjectModelExecutionException: ignored

In [20]:
import layer

my_model = layer.get_model("tomy4reel/aqssh-645h-wver/models/model1:3.4").get_train()

Output()

In [21]:
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = my_model.predict(test_df[cols]) 

In [22]:
submission.head()

,ID,pm2_5
3557,ID_M1T3S50WVB,75.021924
1406,ID_35WTU72HHD,72.925188
925,ID_O7K2PDWG93,69.125007
3432,ID_N9WDA8G4OZ,63.122002
2205,ID_WFYVDUY1LY,79.316093


In [11]:
# Data for new devices
test_df=test_df[test_df['ID'].isin(list(missing['ID'].unique()))]
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)

### Second Model (for new devices only)

In [12]:
@model("model2")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model2 = layer.get_model("tomy4reel/aqssh-645h-wver/models/model2").get_train()
submission2 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model2.predict(test_df[cols])    

Output()

ProjectInitializationException: ignored

In [ ]:
# replace predictions for new devices with predictions from model2
submission=submission2.append(submission)
submission=submission.drop_duplicates('ID')
df1=pd.merge(ss[['ID']],submission,on='ID',how='left')
df1

In [ ]:
# reload datasets
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()

In [ ]:
# re-process data 
# create more features 
cc=['site_longitude','SulphurDioxide_SO2_column_number_density','Cloud_cloud_base_height','NitrogenDioxide_sensor_altitude','SulphurDioxide_sensor_azimuth_angle','SulphurDioxide_sensor_zenith_angle',
    'NitrogenDioxide_sensor_azimuth_angle','NitrogenDioxide_sensor_zenith_angle','Formaldehyde_sensor_azimuth_angle','Formaldehyde_sensor_zenith_angle','Ozone_sensor_azimuth_angle','Ozone_sensor_zenith_angle',
    'Cloud_sensor_azimuth_angle','Cloud_sensor_zenith_angle','CarbonMonoxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle']
train_df.drop(cc,axis=1,inplace=True)
train_df['date']=pd.to_datetime(train_df['date'])
test_df['date']=pd.to_datetime(test_df['date'])
min_date=train_df['date'].min()
train_df['timestamp']=train_df['date']-min_date
test_df['timestamp']=test_df['date']-min_date
train_df['timestamp']=train_df['timestamp'].astype('int')/(24*60*60*(10**9))
test_df['timestamp']=test_df['timestamp'].astype('int')/(24*60*60*(10**9))
train_df['timestamp']=train_df['timestamp'].astype('int')
test_df['timestamp']=test_df['timestamp'].astype('int')
density_amf = list(train_df.filter(regex='density_amf').columns)
density = ['SulphurDioxide_SO2_slant_column_number_density','SulphurDioxide_SO2_column_number_density_15km','CarbonMonoxide_CO_column_number_density','CarbonMonoxide_H2O_column_number_density',
 'NitrogenDioxide_NO2_column_number_density', 'NitrogenDioxide_tropospheric_NO2_column_number_density', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'NitrogenDioxide_NO2_slant_column_number_density', 'Formaldehyde_tropospheric_HCHO_column_number_density',
 'Formaldehyde_HCHO_slant_column_number_density','Ozone_O3_column_number_density','Ozone_O3_slant_column_number_density']
Cloud = list(train_df.filter(regex='Cloud').columns)
Ozone = list(train_df.filter(regex='Ozone').columns)
Carbon = list(train_df.filter(regex='Carbon').columns)
Aerosol = list(train_df.filter(regex='Aerosol').columns)
Sulphur = list(train_df.filter(regex='Sulphur').columns)
Nitrogen = list(train_df.filter(regex='Nitrogen').columns)
Formaldehyde = list(train_df.filter(regex='Formaldehyde').columns)
Weather = ['temp_mean','humidity']
test_df[target]=np.nan
test_df=test_df[train_df.columns]
train_df=train_df.append(test_df)
train_df['density_sum']=train_df[density].sum(axis=1)
train_df['density_amf_sum']=train_df[density_amf].sum(axis=1)
train_df['NO2_sum']=train_df.NitrogenDioxide_NO2_column_number_density+train_df.NitrogenDioxide_tropospheric_NO2_column_number_density
train_df=train_df.sort_values(['device','timestamp'])
# feature generation
for col in Cloud+Ozone+Carbon+Aerosol+Sulphur+Nitrogen+Weather+Formaldehyde:
    train_df[col+'dev']=train_df.groupby('device')[col].transform('mean')
    train_df[col+'diff']=train_df[col]/train_df[col+'dev']
    train_df[col+'std']=train_df.groupby('device')[col].transform('std')
    #train_df[col+'skew']=train_df.groupby('device')[col].transform('skew')
    b=3
    train_df[col+'rollin_mean']=train_df.groupby('device')[col].rolling(b,min_periods=b).mean().values
    #train_df[col+'rollin_std']=train_df.groupby('device')[col].rolling(b,min_periods=b).std().values
    for i in range(1,35,1):
        train_df[col+'change'+str(i)]=train_df.groupby('device')[col].shift(i)
        train_df[col+'change'+str(i)]=train_df[col]/train_df[col+'change'+str(i)]
missing=train_df[train_df.device.isin(['aq_91', 'aq_98'])]
le = LabelEncoder()
train_df = train_df.assign(device = le.fit_transform(train_df["device"]))
drop_cols=["pm2_5", "ID","date"]
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
print('length of features = ',len(cols))

### Third Model

In [ ]:
@model("model3")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model3 = layer.get_model("tomy4reel/aqssh-645h-wver/models/model3").get_train()
submission = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission["pm2_5"] = model3.predict(test_df[cols])    

In [ ]:
# create data for new devices
test_df=test_df[test_df['ID'].isin(list(missing['ID'].unique()))]
drop_cols=test_df.columns[test_df.isnull().all()].tolist()[1:]
cols=list(test_df[cols].drop(drop_cols,axis=1).columns)

### Fourth Model (for new devices only)

In [ ]:
@model("model4")
@pip_requirements(packages=["catboost"])
def train():
    import catboost
    from catboost import CatBoostRegressor
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    est=200
    random_state = 42
    layer.log({"est":est})
    layer.log({"random_state":random_state})
    model=CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=est).fit(train_df[cols],train_df[target])
    importance = model.get_feature_importance()
    importance_df = pd.DataFrame(importance, index=cols, columns=['Importance'])
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    importance_df = importance_df[importance_df['Importance'] > 0]
    importance_df = importance_df.head(10)
    plt.figure(figsize=(8,8))
    plt.xticks(rotation=60, fontsize = 20)
    sns.barplot(y=importance_df.index, x=importance_df['Importance'])
    layer.log({"Feature importance": plt.gcf()})
    return model
layer.run([train],debug=True)
model4 = layer.get_model("tomy4reel/aqssh-645h-wver/models/model4").get_train()
submission4 = pd.DataFrame(data=test_df["ID"], columns = ["ID"])
submission2["pm2_5"] = model4.predict(test_df[cols])    

In [ ]:
# do not replace predictions for new devices with predictions from model4
# take mean instead
submission=submission2.append(submission)
sm=submission.copy()
sm[target]=submission.groupby('ID')['pm2_5'].transform('mean')
df2=sm.drop_duplicates('ID')
df2

In [ ]:
# blend predictions
cols = list(df1.columns)
index_col = list(df1.columns)[0]
df = pd.merge(df1,df2,on=index_col,how='left')
for col in cols[1:]:
    df[col] = (df[col+'_x']*0.5) + (df[col+'_y']*0.5)
df

In [ ]:
df[cols].to_csv('es30.csv',index=False)
df[cols]